In [7]:
# # # make sure to use the latest version of the openai python package
# # %pip install --upgrade --quiet openai
# %pip install python-dotenv

In [1]:
import asyncio
import os
from openai import AsyncOpenAI
import openai
import json
from pprint import pprint

In [2]:
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
org_id = os.getenv("ORG_ID")
project_id = os.getenv("PROJECT_ID")

In [3]:

client = openai.OpenAI(
    api_key=openai_api_key,
    organization=org_id,
    project=project_id,
)

In [8]:
training_file_name = "train_data_v2.jsonl"
validation_file_name = "eval_data_v2.jsonl"

In [12]:
# print the first 5 lines of the training file
!head -n 3 eval_data_v2.jsonl

{"messages": [{"role": "system", "content": "You are an AI assistant tasked with extracting management levels and job titles from a given query. Your goal is to analyze the query, identify relevant key phrases, and categorize them appropriately as either management levels or job titles while generating proper reasoning and thinking step by step. \nYour output format should be like below:\n<rationale>\nProvide reasoning and step-by-step thinking inside here\n</rationale>\n<json_output>\nProvide json output inside here\n</json_output>\n"}, {"role": "user", "content": "User Query: \nInstructions:\n1. Identify key phrases that are management levels or job titles.\n2. Categorize each key phrase as either a management level or a job title.\n3. Use predefined management levels: ['Partners,',' 'Founder or Co-founder,' 'Board of Directors,' 'CSuite/Chiefs,' 'Executive VP or Sr. VP,' 'General Manager,' 'Manager,' 'Head,' 'Senior Partner,' 'Junior Partner,' 'VP,' 'Director,' 'Senior (All Senior-L

In [13]:
def upload_file(file_name: str, purpose: str) -> str:
    with open(file_name, "rb") as file_fd:
        response = client.files.create(file=file_fd, purpose=purpose)
    return response.id


training_file_id = upload_file(training_file_name, "fine-tune")
validation_file_id = upload_file(validation_file_name, "fine-tune")

print("Training file ID:", training_file_id)
print("Validation file ID:", validation_file_id)

Training file ID: file-33CacLrMDhdXJTKMCxjqQ8
Validation file ID: file-KUy5fGEYjtjXZ7Ev4N6uQF


In [14]:
# MODEL = "gpt-4o-mini-2024-07-18"
FT_MODEL_V1_338 = "ft:gpt-4o-mini-2024-07-18:qlu:ner-v1-338-training-examples-08-01-2024-3-epochs:AnSGTc0m:ckpt-step-676"

response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model=FT_MODEL_V1_338,
    suffix="ner_v2:49-improved-training-examples-09-01-2024:2-epochs",
    method={
        "type": "supervised",
        "supervised": {
        "hyperparameters": {
            "batch_size": "auto",
            "learning_rate_multiplier": "auto",
            "n_epochs": "2",
            }
        }
    }
)

job_id = response.id

print("Job ID:", response.id)
print("Status:", response.status)

Job ID: ftjob-K0cQAjM657O6SRqPv8i4vVRN
Status: validating_files
